In [1]:
from langchain.document_loaders import TextLoader
loader = TextLoader('qa_clean.txt')
documents = loader.load()

In [2]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1500, separator="\n")
chunks = text_splitter.split_documents(documents)

Created a chunk of size 2250, which is longer than the specified 1500


In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs)

/Users/jiaying/anaconda/anaconda3/envs/LLMsearch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
from langchain.vectorstores import Chroma
persist_directory = 'db'
vectordb = Chroma.from_documents(documents=chunks, embedding=embedding, persist_directory=persist_directory)

In [5]:
from langchain_community.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q4_K_M.gguf",
    temperature=0.75,
    n_ctx=2048,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 

In [6]:
from langchain.chains import RetrievalQA
retriever = vectordb.as_retriever(search_kwargs={"k":3})

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [8]:
query = "how do I cancel a card in Nordea"
qa.invoke(query)



> Entering new RetrievalQA chain...


Llama.generate: prefix-match hit


 I don't know. I'm just an AI and do not have access to personal financial information or the ability to cancel a card on behalf of a user. Canceling a card is a personal decision that should be made after consulting with a financial advisor or contacting the bank directly.


llama_print_timings:        load time =   13072.33 ms
llama_print_timings:      sample time =      16.30 ms /    65 runs   (    0.25 ms per token,  3986.75 tokens per second)
llama_print_timings: prompt eval time =   48178.73 ms /   524 tokens (   91.94 ms per token,    10.88 tokens per second)
llama_print_timings:        eval time =    9003.97 ms /    64 runs   (  140.69 ms per token,     7.11 tokens per second)
llama_print_timings:       total time =   57562.13 ms /   588 tokens



> Finished chain.


{'query': 'how do I cancel a card in Nordea',
 'result': " I don't know. I'm just an AI and do not have access to personal financial information or the ability to cancel a card on behalf of a user. Canceling a card is a personal decision that should be made after consulting with a financial advisor or contacting the bank directly."}